In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Aug 16 17:10:53 2020

@author: wanxiang.shen@u.nus.edu
"""

import warnings, os
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggModel, loadmap
from aggmap import show
np.random.seed(666) #just for reaptable results

def prc_auc_score(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc


def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])
    auc = roc_auc_score(y_true, y_score)

    print('acc: %.3f, roc-auc: %.3f' % (acc,auc))

    return acc, auc

In [2]:
label = pd.read_excel('./subject_info.xlsx')
data = pd.read_excel('./data_species.xlsx', index_col = 0)

In [3]:
label.groupby('Group').size()

Group
HS               30
Healthy         127
MP               40
Stage_0          27
Stage_III_IV     54
Stage_I_II       69
dtype: int64

## Model-1: Healthy vs. Stage_I_II:

In [8]:
stage = 'Stage_I_II'
dfy = label[label.Group.isin(['Healthy', stage])].set_index('Subject_ID')[['Group']] #, 'Stage_I_II', 'Stage_III_IV'
idx=dfy[dfy['Group'] == stage].index
s = data.loc[idx].mean()
flist = s[s > 1e-7].index # the same as the paper used
data1 = data[flist]
data1.shape

(347, 4340)

In [9]:
Y = pd.get_dummies(dfy['Group']).values
dfx = data1.loc[dfy.index]
dfx = np.log(dfx + 1e-8) #log transoform

In [10]:
dfy.shape, dfx.shape

((196, 1), (196, 4340))

In [11]:
mp = AggMap(dfx, metric = 'correlation')
mp.save('./%s.mp' % stage)
mp = loadmap('./%s.mp' % stage)

2020-12-13 13:19:05,199 - INFO - [bidd-aggmap] - Calculating distance ...
2020-12-13 13:19:06,421 - INFO - [bidd-aggmap] - the number of process is 16


100%|##########| 4340/4340 [00:05<00:00, 796.21it/s]


In [12]:
mp.fit(cluster_channels = 5, verbose = 0, var_thr = 0)

2020-12-13 13:24:06,343 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2020-12-13 13:24:21,199 - INFO - [bidd-aggmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2020-12-13 13:25:14,023 - INFO - [bidd-aggmap] - Finished


In [13]:
mp.fmap_shape

(66, 66)

In [14]:
mp.plot_scatter()

2020-12-13 13:25:14,050 - INFO - [bidd-aggmap] - generate file: ./feature points_4340_correlation_umap_scatter
2020-12-13 13:25:14,134 - INFO - [bidd-aggmap] - save html file to ./feature points_4340_correlation_umap_scatter


In [15]:
X = mp.batch_transform(dfx.values, scale_method = 'standard')

100%|##########| 196/196 [00:14<00:00, 13.52it/s]


In [16]:
class_ = pd.get_dummies(dfy['Group']).columns
class_

Index(['Healthy', 'Stage_I_II'], dtype='object')

In [17]:
outer_fold = 10
repeat_seeds = [8, 16, 32, 64, 128]#5 repeats random seeds 8, 16, 32, 64, 128

run_all_res = []
run_all = []
for repeat_seed in repeat_seeds: 
    
    outer = StratifiedKFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(X, Y[:,0])

    
    run_one_res = []
    for i, idx in enumerate(outer_idx):
        
        fold_num = "fold_%s" % str(i).zfill(2) 
        
        train_idx, test_idx = idx
        
        testY = Y[test_idx]
        testX = X[test_idx]
        
        trainX = X[train_idx]
        trainY = Y[train_idx]

        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))
        
        clf = AggModel.MultiClassEstimator(epochs = 50, gpuid = 3, conv1_kernel_size = 3,
                                           batch_size = 2, verbose = 0) #
        clf.fit(trainX, trainY ) 

        ## save model
        clf._model.save(os.path.join('./Stage_I_II_model', 'seed_%s_fold_%s.h5' % (repeat_seed, i)))
    
        y_true = testY[:,1] #Stage_0
        y_score = clf.predict_proba(testX)[:,1]
        
        dfr = pd.DataFrame([y_true, y_score]).T
        dfr.columns = ['y_true', 'y_score']
        dfr.index = dfy.iloc[test_idx].index
        
        acc, auc = score(dfr)
        
        run_one_res.append(dfr)
        
        run_all.append([repeat_seed, i, auc])
    run_all_res.append(pd.concat(run_one_res))


 input train and test X shape is (176, 66, 66, 5), (20, 66, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, gpuid='3')
acc: 0.900, roc-auc: 0.934

 input train and test X shape is (176, 66, 66, 5), (20, 66, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, gpuid='3')
acc: 0.750, roc-auc: 0.758

 input train and test X shape is (176, 66, 66, 5), (20, 66, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, gpuid='3')
acc: 0.800, roc-auc: 0.813

 input train and test X shape is (176, 66, 66, 5), (20, 66, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, gpuid='3')
acc: 0.900, roc-auc: 0.868

 input train and test X shape is (176, 66, 66, 5), (20, 66, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, gpuid='3')
acc: 0.850, roc-auc: 0.923

 input train and test X shape is (176, 66, 66, 5), (20, 66, 66, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=3, epochs=50, 

In [18]:
results = []
for dfr, repeat_seed in zip(run_all_res,repeat_seeds):

    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()


    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])

    sensitivity = tp / sum([tp, fn])
    specificity = tn / sum([tn, fp])

    prc_auc = prc_auc_score(y_true, y_score)
    roc_auc = roc_auc_score(y_true, y_score)

    precision = tp / sum([tp, fp])
    recall =  tp / sum([tp, fn]) #equals to sensitivity

    res = {
           'repeat_seed':repeat_seed,

           'accuracy':acc, 

           'prc_auc':prc_auc, 
           'roc_auc':roc_auc,

           'sensitivity': sensitivity, 
           'specificity': specificity,

           'precision':precision,
           'recall':recall,
           'F1': 2*precision*sensitivity/(precision+sensitivity)
          }
    
    results.append(res)

In [19]:
pd.DataFrame(results).roc_auc.mean()

0.8652288029213737

In [21]:
pd.DataFrame(results)

,repeat_seed,accuracy,prc_auc,roc_auc,sensitivity,specificity,precision,recall,F1
0,8,0.841837,0.826499,0.858382,0.652174,0.944882,0.865385,0.652174,0.743802
1,16,0.831633,0.829346,0.857241,0.594203,0.960630,0.891304,0.594203,0.713043
2,32,0.821429,0.823541,0.861235,0.608696,0.937008,0.840000,0.608696,0.705882
3,64,0.836735,0.838652,0.873902,0.637681,0.944882,0.862745,0.637681,0.733333
4,128,0.846939,0.847579,0.875385,0.666667,0.944882,0.867925,0.666667,0.754098
